In [201]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/home/sharanbalakrishnan/Desktop/InternProj/data/samsung_results.csv')

In [3]:
df1 = pd.read_csv('/home/sharanbalakrishnan/Desktop/InternProj/data/realme_results.csv')

In [210]:
data = pd.read_csv('/home/sharanbalakrishnan/Desktop/InternProj/merged_df_one.csv')

In [48]:
'''import re

colors = ['White', 'Black', 'Grey', 'Blue', 'Purple', 'Gold', 'Silver', 'Orange', 'Pearl', 'Cream', 'Carbon', 'Green', 'Yellow']

def extract_color(string):
    pattern = '|'.join(colors)
    match = re.search(pattern, string, flags=re.IGNORECASE)
    if match:
        return match.group(0)
    else:
        return 'Grey'

# Example usage:
data['Color'] = data['Model'].apply(extract_color)'''


"import re\n\ncolors = ['White', 'Black', 'Grey', 'Blue', 'Purple', 'Gold', 'Silver', 'Orange', 'Pearl', 'Cream', 'Carbon', 'Green', 'Yellow']\n\ndef extract_color(string):\n    pattern = '|'.join(colors)\n    match = re.search(pattern, string, flags=re.IGNORECASE)\n    if match:\n        return match.group(0)\n    else:\n        return 'Grey'\n\n# Example usage:\ndata['Color'] = data['Model'].apply(extract_color)"

In [84]:
def clean_data(data):
    data['Brand'] = data['Model'].str.split().str[0]
    data['Brand'] = data['Brand'].str.lower().replace({'mi3':'Mi', 'mi4':'Mi', 'mi':'Mi'})
    data['Color'] = data['Model'].str.extract(r'\((.*?)\)')[0].str.split().str[-1].str.lower()
    data["Display_Size"] = data["Display"].str.extract(r'(\d+\.\d+)\s*inch', expand=False)
    data['Display_Type'] = data['Display'].str.extract(r'(\S+)\s*Display')
    data = data.drop_duplicates()
    data['Front_Cam'] = data['Camera'].str.split('|').str[1].str.extract('(\d+)(?=MP)')
    data['Rear_Cam'] = data['Camera'].str.extract('^(\d+)MP')
    data['ROM'] = data['Storage'].str.extract('(\d+)\s*(?:GB|MB)?')
    data = data.drop(index=data[(data['Brand'] == 'note') | (data['Brand'] == '�9a')].index)
    data = data.drop(index=data[(data['Display_Type'] == 'inch)') | (data['Display_Type'] == 'NA')].index)
    data = data.drop(index=data[(data['Front_Cam'] == '0')].index)
    data['Color'] = data['Color'].replace(['WHITE', 'White', 'Cream'], 'White')
    data['Display_Size'] = data['Display_Size'].fillna(method='ffill')
    data.dropna(axis=0, inplace=True)
    data['Front_Cam'] = data['Front_Cam'].astype(float)
    data['Rear_Cam'] = data['Rear_Cam'].astype(float)
    data['Price'] = data['Price'].str.replace('₹', '').str.replace(',', '')
    data['Price'] = data['Price'].astype(float)
    data['Display_Size'] = data['Display_Size'].astype(float)
    data = data.drop(index=data[(data['RAM'] == '1') | (data['ROM'] == '1')].index)
    data = data.drop(index=data[data['ROM'] == '8'].index)

    colors = ['White', 'Black', 'Grey', 'Blue', 'Purple', 'Gold', 'Silver', 'Orange', 'Pearl', 'Cream', 'Carbon', 'Green', 'Yellow']

    def extract_color(string):
        pattern = '|'.join(colors)
        match = re.search(pattern, string, flags=re.IGNORECASE)
        if match:
            return match.group(0)
        else:
            return 'Grey'

    data['Color'] = data['Model'].apply(extract_color)
    
    return data


In [174]:
#print(int(data[data['Brand'] == 'samsung'].Storage.value_counts().head(1)))

print(data[data['Brand'] == 'samsung'].Ram.value_counts().index[0])

print(data[data['Brand'] == 'samsung'].Storage.value_counts().index[0])

print(data[data['Brand'] == 'samsung'].Display_Type.value_counts().index[0])

print(data[data['Brand'] == 'samsung'].iloc[: , 11:13].value_counts().index[0])

print(data[data['Brand'] == 'samsung'].Color.describe().top)

print(float(data[data['Brand'] == 'samsung'].Price.max()))

print(float(data[data['Brand'] == 'samsung'].Price.min()))

print(float(data[data['Brand'] == 'samsung'].Price.mean()))

8 GB RAM 
128 GB RO
HD+
(8.0, 50.0)
Black
79999.0
8990.0
32998.4375


In [212]:
def brand_statistics(brand_name, data):
    brand_data = data[data['Brand'] == brand_name]
    
    ram_mode = brand_data.Ram.mode()[0]
    storage_mode = brand_data.Storage.mode()[0]
    display_type_mode = brand_data.Display_Type.mode()[0]
    camera_mode = brand_data.iloc[:, 11:13].mode().iloc[0].tolist()
    color_top = brand_data.Color.describe().top
    price_max = float(brand_data.Price.max())
    price_min = float(brand_data.Price.min())
    price_mean = float(brand_data.Price.mean())
    
    print(f"RAM mode: {ram_mode}")
    print(f"Storage mode: {storage_mode}")
    print(f"Display type mode: {display_type_mode}")
    print(f"Camera mode: {camera_mode}")
    print(f"Color with highest frequency: {color_top}")
    print(f"Maximum price: {price_max}")
    print(f"Minimum price: {price_min}")
    print(f"Mean price: {price_mean}")

In [213]:
brand_statistics('apple' , data)

RAM mode: 128 GB RO
Storage mode: 128 GB ROM
Display type mode: XDR
Camera mode: ['XDR', 12.0]
Color with highest frequency: Grey
Maximum price: 178099.0
Minimum price: 14999.0
Mean price: 83784.16513761468


In [214]:
import pandas as pd

def compare_brands(brand1, brand2, data):
    brand1_data = data[data['Brand'] == brand1]
    brand2_data = data[data['Brand'] == brand2]
    
    stats = {}
    stats[f"{brand1} RAM"] = brand1_data.Ram.mode().values[0]
    stats[f"{brand2} RAM"] = brand2_data.Ram.mode().values[0]
    stats[f"{brand1} Storage"] = brand1_data.Storage.mode().values[0]
    stats[f"{brand2} Storage"] = brand2_data.Storage.mode().values[0]
    stats[f"{brand1} Display Type"] = brand1_data.Display_Type.mode().values[0]
    stats[f"{brand2} Display Type"] = brand2_data.Display_Type.mode().values[0]
    stats[f"{brand1} Rear Cam"] = brand1_data.Rear_Cam.mean()
    stats[f"{brand2} Rear Cam"] = brand2_data.Rear_Cam.mean()
    stats[f"{brand1} Price"] = brand1_data.Price.mean()
    stats[f"{brand2} Price"] = brand2_data.Price.mean()
    
    df = pd.DataFrame(stats.items(), columns=['Metric', 'Value'])
    
    return df


In [215]:
df_comparison = compare_brands('samsung', 'xiaomi', data)
print(df_comparison)


                 Metric         Value
0           samsung RAM     8 GB RAM 
1            xiaomi RAM     6 GB RAM 
2       samsung Storage     128 GB RO
3        xiaomi Storage     128 GB RO
4  samsung Display Type           HD+
5   xiaomi Display Type        AMOLED
6      samsung Rear Cam         42.75
7       xiaomi Rear Cam          92.0
8         samsung Price    32998.4375
9          xiaomi Price  24960.636364


In [216]:
import pandas as pd

def brand_comparison(brand1, brand2, data):
    brand1_stats = [brand1, 
                    data[data['Brand'] == brand1].Ram.value_counts().index[0], 
                    data[data['Brand'] == brand1].Storage.value_counts().index[0], 
                    data[data['Brand'] == brand1].Display_Type.value_counts().index[0], 
                    data[data['Brand'] == brand1].iloc[: , 11:13].value_counts().index[0], 
                    data[data['Brand'] == brand1].Color.describe().top, 
                    float(data[data['Brand'] == brand1].Price.max()), 
                    float(data[data['Brand'] == brand1].Price.min()), 
                    float(data[data['Brand'] == brand1].Price.mean())]
    
    brand2_stats = [brand2, 
                    data[data['Brand'] == brand2].Ram.value_counts().index[0], 
                    data[data['Brand'] == brand2].Storage.value_counts().index[0], 
                    data[data['Brand'] == brand2].Display_Type.value_counts().index[0], 
                    data[data['Brand'] == brand2].iloc[: , 11:13].value_counts().index[0], 
                    data[data['Brand'] == brand2].Color.describe().top, 
                    float(data[data['Brand'] == brand2].Price.max()), 
                    float(data[data['Brand'] == brand2].Price.min()), 
                    float(data[data['Brand'] == brand2].Price.mean())]
    
    df = pd.DataFrame([brand1_stats, brand2_stats], 
                      columns=['Brand', 'RAM', 'Storage', 'Display_Type', 'Camera', 'Color', 'Max_Price', 'Min_Price', 'Mean_Price'])
    return df.set_index('Brand')


In [217]:
brand_comparison('samsung', 'xiaomi', data)

,RAM,Storage,Display_Type,Camera,Color,Max_Price,Min_Price,Mean_Price
Brand,,,,,,,,
samsung,8 GB RAM,128 GB RO,HD+,"(HD+, 8.0)",Black,79999.0,8990.0,32998.437500
xiaomi,8 GB RAM,128 GB RO,AMOLED,"(AMOLED, 16.0)",Black,26999.0,20990.0,24960.636364


In [192]:
data['RAM'] = data['Ram'].str.extract('(\d+)\s*(?:GB|MB)?')

In [193]:
data

,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
23,"SAMSUNG Galaxy A23 5G (Orange, 128 GB)",6 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP + 5MP | 8MP Front Camera,5000 mAh Lithium Ion Battery,22999.0,samsung,Orange,6.60,HD+,8.0,50.0,128,6
69,"SAMSUNG Galaxy A14 5G (Black, 128 GB)",6 GB RAM,128 GB RO,16.76 cm (6.6 inch) HD+ Display,50MP + 2MP | 13MP Front Camera,5000 mAh Lithium Ion Battery,18999.0,samsung,Black,6.60,HD+,13.0,50.0,128,6
115,"SAMSUNG Galaxy A23 5G (Silver, 128 GB)",8 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP + 5MP | 8MP Front Camera,5000 mAh Lithium Ion Battery,24999.0,samsung,Silver,6.60,HD+,8.0,50.0,128,8
138,"SAMSUNG Galaxy A23 (Blue, 128 GB)",8 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP Rear Camera | 8MP Front Camera,5000 mAh Li-Ion Battery,18999.0,samsung,Blue,6.60,HD+,8.0,50.0,128,8
161,"SAMSUNG Galaxy S23 5G (Green, 256 GB)",8 GB RAM,256 GB RO,15.49 cm (6.1 inch) Full HD+ Display,50MP + 10MP + 12MP | 12MP Front Camera,3900 mAh Lithium Ion Battery,79999.0,samsung,Green,6.10,HD+,12.0,50.0,256,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2397,"OPPO F19 (Space Silver, 128 GB)",6 GB RAM,128 GB RO,16.33 cm (6.43 inch) Full HD+ Display,48MP + 2MP + 2MP | 16MP Front Camera,5000 mAh Battery,14990.0,oppo,Silver,6.43,HD+,16.0,48.0,128,6
2398,"REDMI 9i Sport (Metallic Blue, 64 GB)",4 GB RAM,64 GB ROM,16.59 cm (6.53 inch) HD+ Display,13MP Rear Camera | 5MP Front Camera,5000 mAh Li-Polymer Battery,7699.0,redmi,Blue,6.53,HD+,5.0,13.0,64,4
2399,"vivo T1 44W (Ice Dawn, 128 GB)",8 GB RAM,128 GB RO,16.36 cm (6.44 inch) Full HD+ AMOLED Display,50MP + 2MP + 2MP | 16MP Front Camera,5000 mAh Lithium Battery,17999.0,vivo,Grey,6.44,AMOLED,16.0,50.0,128,8
2400,"vivo V21 5G (Sunset Dazzle, 256 GB)",8 GB RAM,256 GB RO,16.36 cm (6.44 inch) Full HD+ Display,64MP + 8MP + 2MP | 44MP Front Camera,4000 mAh Lithium-ion Battery,30990.0,vivo,Grey,6.44,HD+,44.0,64.0,256,8


In [213]:
print(data[data['RAM'] == '6'].Brand.value_counts().index[0])

print(data[data['RAM'] == '6'].Storage.value_counts().index[0])

print(float(data[data['RAM'] == '6'].Price.mean()))

print(float(data[data['RAM'] == '6'].Price.max()))



redmi
128 GB RO
19222.919811320753
37999.0


In [234]:
#data[(data['RAM'] == '6')](data['Price'] == data.Price.max())

#print(data[data['Price'] ==  data.Price.max()].Brand.value_counts().index[0])

print(data[data['Price'] ==  data.Price.min()].Brand.value_counts().index[0])

print(data[data['Price'] ==  data.Price.min()].iloc[: , 11:13].value_counts().index[0])

print(data[data['Price'] <=  data.Price.mean()].Brand.value_counts().index[0:3])

print(data[data['Price'] >  data.Price.mean()].Brand.value_counts().index[0:3])

realme
(5.0, 8.0)
Index(['realme', 'redmi', 'vivo'], dtype='object', name='Brand')
Index(['apple', 'realme', 'vivo'], dtype='object', name='Brand')


In [23]:
print(data[data['RAM'] == 8].Price.min())

print(data[data['RAM'] == 8].Price.max())

print(data[(data['RAM'] == 8) & (data['Price'] == data[data['RAM'] == 8].Price.max())].Brand.value_counts().index[0])

print(data[(data['RAM'] == 8) & (data['Price'] == data[data['RAM'] == 8].Price.min())].Brand.value_counts().index[0])

14999.0
89999.0
oppo
apple


In [36]:
data[(data['RAM'] == 256) & (data['Price'] == data[data['RAM'] == 256].Price.min())]

,Unnamed: 0,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
719,1520,"APPLE iPhone SE (White, 256 GB)",256 GB RO,256 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,A13 Bionic Chip with 3rd Gen Neural Engine Pro...,40990.0,apple,White,4.7,HD,7.0,12.0,256,256
771,1578,"Apple iPhone SE (Black, 256 GB) (Includes EarP...",256 GB RO,256 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,A13 Bionic Chip with 3rd Gen Neural Engine Pro...,40990.0,apple,Black,4.7,HD,7.0,12.0,256,256
833,1674,"APPLE iPhone SE (Black, 256 GB)",256 GB RO,256 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,A13 Bionic Chip with 3rd Gen Neural Engine Pro...,40990.0,apple,Black,4.7,HD,7.0,12.0,256,256
887,1758,"Apple iPhone SE (White, 256 GB) (Includes EarP...",256 GB RO,256 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,A13 Bionic Chip with 3rd Gen Neural Engine Pro...,40990.0,apple,White,4.7,HD,7.0,12.0,256,256


In [40]:
data.Ram.unique()

array(['6 GB RAM ', '8 GB RAM ', '3 GB RAM ', '4 GB RAM ', '2 GB RAM ',
       '12 GB RAM ', '1 GB RAM ', '128 GB RO', '64 GB RO', '256 GB RO',
       '512 GB RO', '1 TB RO', '32 GB RO', '16 GB RO', '8 GB RO'],
      dtype=object)

In [41]:
data.Model[1]

'SAMSUNG Galaxy A14 5G (Black, 128 GB)'

In [63]:
lis = data[((data.RAM) == (data.ROM)) | (data.RAM > 12)].Model.unique()

In [69]:
lis[1][:30]

'APPLE iPhone 11 (Black, 128 GB'

In [71]:
len(lis[1])

31

In [97]:
new = []
for i in range(len(lis)):

    for j in range(len(lis[i])):
        if lis[i][j]=='(':
            new.append(lis[i][:j])  
            continue
             
        
            
         
    

In [ ]:
#import pandas as pd

# create a dictionary with iPhone models as keys and RAM specifications as values
ram_specs = {
    'iPhone 11': '4 GB',
    'iPhone 12': '4 GB',
    'iPhone 13': '6 GB',
    'iPhone SE': '3 GB',
    'iPhone XR': '3 GB',
    'iPhone XS': '4 GB',
    'iPhone X': '3 GB',
    'iPhone 8 Plus': '3 GB',
    'iPhone 8': '2 GB',
    'iPhone 7 Plus': '3 GB',
    'iPhone 7': '2 GB',
    'iPhone 6s Plus': '2 GB',
    'iPhone 6s': '2 GB',
    'iPhone 6 Plus': '1 GB',
    'iPhone 6': '1 GB',
    'iPhone SE (2nd generation)': '3 GB',
    'iPhone 11 Pro': '4 GB',
    'iPhone 11 Pro Max': '4 GB',
    'iPhone XS Max': '4 GB',
    'iPhone XR': '3 GB',
    'iPhone X': '3 GB',
    'iPhone 12 mini': '4 GB',
    'iPhone 12 Pro': '6 GB',
    'iPhone 12 Pro Max': '6 GB',
    'iPhone 13 mini': '4 GB',
    'iPhone 13 Pro': '8 GB',
    'iPhone 13 Pro Max': '8 GB',
    'iPhone 14': '8 GB',
    'iPhone 14 mini': '6 GB',
    'iPhone 14 Pro': '8 GB',
    'iPhone 14 Pro Max': '8 GB'
}



# extract the iPhone model name from the 'Model' column
#data['iPhone_Model'] = df['Model'].str.extract('(iPhone [^ ]+)', expand=False)

# map the iPhone model to its RAM specification using the 'ram_specs' dictionary
df['RAM'] = df['iPhone Model'].map(ram_specs)

# display the resulting dataframe
print(df)


In [130]:
ram_specs = {
    'iPhone 11': 4 ,
    'iPhone 12': 4,
    'iPhone 13': 6 ,
    'iPhone SE': 3 ,
    'iPhone XR': 3 ,
    'iPhone XS': 4 ,
    'iPhone X': 3 ,
    'iPhone 8 Plus': 3,
    'iPhone 8': 2 ,
    'iPhone 7 Plus': 3 ,
    'iPhone 7': 2 ,
    'iPhone 6s Plus': 2,
    'iPhone 6s': 2 ,
    'iPhone 6 Plus': 1 ,
    'iPhone 6': 1 ,
    'iPhone SE (2nd generation)': 3 ,
    'iPhone 11 Pro': 4 ,
    'iPhone 11 Pro Max': 4 ,
    'iPhone XS Max': 4 ,
    'iPhone XR': 3 ,
    'iPhone X': 3 ,
    'iPhone 12 mini': 4,
    'iPhone 12 Pro': 6 ,
    'iPhone 12 Pro Max': 6 ,
    'iPhone 13 mini': 4 ,
    'iPhone 13 Pro': 8 ,
    'iPhone 13 Pro Max': 8 ,
    'iPhone 14': 8 ,
    'iPhone 14 mini': 6 ,
    'iPhone 14 Pro': 8 ,
    'iPhone 14 Pro Max': 8 
}


In [ ]:
def brand_comparison(brand1, brand2, data):
    
    brand1_stats = [brand1, 
                    data[data['Brand'] == brand1].Ram.value_counts().index[0], 
                    data[data['Brand'] == brand1].Storage.value_counts().index[0], 
                    data[data['Brand'] == brand1].Display_Type.value_counts().index[0], 
                    data[data['Brand'] == brand1].iloc[: , 11:13].value_counts().index[0], 
                    data[data['Brand'] == brand1].Color.describe().top, 
                    float(data[data['Brand'] == brand1].Price.max()), 
                    float(data[data['Brand'] == brand1].Price.min()), 
                    float(data[data['Brand'] == brand1].Price.mean())]
    
    brand2_stats = [brand2, 
                    data[data['Brand'] == brand2].Ram.value_counts().index[0], 
                    data[data['Brand'] == brand2].Storage.value_counts().index[0], 
                    data[data['Brand'] == brand2].Display_Type.value_counts().index[0], 
                    data[data['Brand'] == brand2].iloc[: , 11:13].value_counts().index[0], 
                    data[data['Brand'] == brand2].Color.describe().top, 
                    float(data[data['Brand'] == brand2].Price.max()), 
                    float(data[data['Brand'] == brand2].Price.min()), 
                    float(data[data['Brand'] == brand2].Price.mean())]
    
    df = pd.DataFrame([brand1_stats, brand2_stats], 
                      columns=['Brand', 'RAM', 'Storage', 'Display_Type', 'Camera', 'Color', 'Max_Price', 'Min_Price', 'Mean_Price'])
    return df.set_index('Brand')

In [233]:
data.Model.values[10:24]


array(['SAMSUNG Galaxy S23 5G (Lavender, 256 GB)',
       'SAMSUNG Galaxy M11 (Black, 32 GB)',
       'SAMSUNG Galaxy A04e (Copper, 64 GB)',
       'SAMSUNG Galaxy F22 (Denim Black, 64 GB)',
       'SAMSUNG Galaxy A13 (Blue, 64 GB)',
       'SAMSUNG Galaxy A23 (Black, 128 GB)',
       'realme C33 (Aqua Blue, 64 GB)',
       'realme Narzo 50 (Speed Black, 128 GB)',
       'realme 10 Pro+ 5G (Nebula Blue, 128 GB)',
       'realme 10 Pro+ 5G (Nebula Blue, 256 GB)',
       'realme 10 Pro+ 5G (Dark Matter, 128 GB)',
       'realme 9i 5G (Rocking Black, 64 GB)',
       'realme 10 Pro+ 5G (Nebula Blue, 128 GB)',
       'realme 10 Pro+ 5G (Dark Matter, 128 GB)'], dtype=object)

In [239]:
data.loc[data['Model'].str.contains('iPhone'), 'Model'] = data['Model'].str.replace(r'APPLE\s?iPhone\s?\d{1,2}\s?(Plus|Pro Max|Pro)?\s?\(.*\)', 'iPhone')


In [242]:
df = data.copy(deep = True)

In [243]:
df.loc[df['Model'].str.contains('iPhone'), 'Model'] = df.loc[df['Model'].str.contains('iPhone'), 'Model'].str.replace(r'APPLE\s?iPhone\s?\d{1,2}\s?(Plus|Pro Max|Pro)?\s?\(.*\)', 'iPhone')


In [250]:
def extract_iphone_model_name(s):
    if 'iPhone' in s:
        return s.split('iPhone')[-1].split('(')[0].strip()
    else:
        return s

# Apply the transformation
data['Model'] = data['Model'].apply(extract_iphone_model_name)

In [253]:
data[data['Brand'] == 'apple'].Model.unique()

array(['14', '11', '13', '14 Plus', '12', '14 Pro', '14 Pro Max',
       '13 Pro', '13 Pro Max', '12 mini', '12 Pro', '13 mini',
       '11 Pro Max', '6s', 'SE', '6', '8 Plus', '8', 'XS Max', '5s',
       '6s Plus', '11 Pro', 'XS', '7', '6 Plus', '7 Plus', 'SE 3rd Gen',
       '5C', '12 Pro Max', 'X'], dtype=object)

In [254]:
ram_dict = {
    '14': 8,
    '11': 4,
    '13': 4,
    '14 Plus': 8,
    '12': 4,
    '14 Pro': 8,
    '14 Pro Max': 8,
    '13 Pro': 6,
    '13 Pro Max': 6,
    '12 mini': 4,
    '12 Pro': 6,
    '13 mini': 4,
    '11 Pro Max': 4,
    '6s': 2,
    'SE': 2,
    '6': 1,
    '8 Plus': 3,
    '8': 2,
    'XS Max': 4,
    '5s': 1,
    '6s Plus': 3,
    '11 Pro': 4,
    'XS': 4,
    '7': 2,
    '6 Plus': 1,
    '7 Plus': 3,
    'SE ': 2,
    '5C': 1,
    '12 Pro Max': 6,
    'X': 3
}



In [255]:
data.loc[data['Model'].isin(ram_dict.keys()), 'RAM'] = data.loc[data['Model'].isin(ram_dict.keys()), 'Model'].map(ram_dict)



In [257]:
data[data['Brand'] == 'apple'].RAM.unique()

array([  8,   4,   6,   2,   1,   3, 256, 128,  64])

In [258]:
data[(data['Brand'] == 'apple') & (data.RAM > 8)]

,Unnamed: 0,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
858,1709,SE 3rd Gen,256 GB RO,256 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",46599.0,apple,Grey,4.7,HD,7.0,12.0,256,256
876,1736,SE 3rd Gen,128 GB RO,128 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",37999.0,apple,Grey,4.7,HD,7.0,12.0,128,128
892,1765,SE 3rd Gen,64 GB RO,64 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",32699.0,apple,Grey,4.7,HD,7.0,12.0,64,64
903,1778,SE 3rd Gen,128 GB RO,128 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",37999.0,apple,Grey,4.7,HD,7.0,12.0,128,128
921,1806,SE 3rd Gen,128 GB RO,128 GB ROM,11.94 cm (4.7 inch) Retina HD Display,12MP Rear Camera | 7MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",37999.0,apple,Grey,4.7,HD,7.0,12.0,128,128


In [261]:
data = data.drop(data[(data['Brand'] == 'apple') & (data.RAM > 8)].index)


In [263]:
data.RAM.unique()

array([ 6,  8,  3,  4,  2, 12,  1])

In [273]:
data.to_csv('merged_latest.csv')

In [274]:
data

,Unnamed: 0,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
0,23,"SAMSUNG Galaxy A23 5G (Orange, 128 GB)",6 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP + 5MP | 8MP Front Camera,5000 mAh Lithium Ion Battery,22999.0,samsung,Orange,6.60,HD+,8.0,50.0,128,6
1,69,"SAMSUNG Galaxy A14 5G (Black, 128 GB)",6 GB RAM,128 GB RO,16.76 cm (6.6 inch) HD+ Display,50MP + 2MP | 13MP Front Camera,5000 mAh Lithium Ion Battery,18999.0,samsung,Black,6.60,HD+,13.0,50.0,128,6
2,115,"SAMSUNG Galaxy A23 5G (Silver, 128 GB)",8 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP + 5MP | 8MP Front Camera,5000 mAh Lithium Ion Battery,24999.0,samsung,Silver,6.60,HD+,8.0,50.0,128,8
3,138,"SAMSUNG Galaxy A23 (Blue, 128 GB)",8 GB RAM,128 GB RO,16.76 cm (6.6 inch) Full HD+ Display,50MP Rear Camera | 8MP Front Camera,5000 mAh Li-Ion Battery,18999.0,samsung,Blue,6.60,HD+,8.0,50.0,128,8
4,161,"SAMSUNG Galaxy S23 5G (Green, 256 GB)",8 GB RAM,256 GB RO,15.49 cm (6.1 inch) Full HD+ Display,50MP + 10MP + 12MP | 12MP Front Camera,3900 mAh Lithium Ion Battery,79999.0,samsung,Green,6.10,HD+,12.0,50.0,256,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,2397,"OPPO F19 (Space Silver, 128 GB)",6 GB RAM,128 GB RO,16.33 cm (6.43 inch) Full HD+ Display,48MP + 2MP + 2MP | 16MP Front Camera,5000 mAh Battery,14990.0,oppo,Silver,6.43,HD+,16.0,48.0,128,6
1243,2398,"REDMI 9i Sport (Metallic Blue, 64 GB)",4 GB RAM,64 GB ROM,16.59 cm (6.53 inch) HD+ Display,13MP Rear Camera | 5MP Front Camera,5000 mAh Li-Polymer Battery,7699.0,redmi,Blue,6.53,HD+,5.0,13.0,64,4
1244,2399,"vivo T1 44W (Ice Dawn, 128 GB)",8 GB RAM,128 GB RO,16.36 cm (6.44 inch) Full HD+ AMOLED Display,50MP + 2MP + 2MP | 16MP Front Camera,5000 mAh Lithium Battery,17999.0,vivo,Grey,6.44,AMOLED,16.0,50.0,128,8
1245,2400,"vivo V21 5G (Sunset Dazzle, 256 GB)",8 GB RAM,256 GB RO,16.36 cm (6.44 inch) Full HD+ Display,64MP + 8MP + 2MP | 44MP Front Camera,4000 mAh Lithium-ion Battery,30990.0,vivo,Grey,6.44,HD+,44.0,64.0,256,8


In [271]:
data[(data.RAM == 6) & (data.Price == data[(data.RAM == 6)].Price.min())]

,Unnamed: 0,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
1023,2045,"REDMI 9 Activ (Coral Green, 128 GB)",6 GB RAM,128 GB RO,16.59 cm (6.53 inch) HD+ Display,13MP + 2MP | 5MP Front Camera,5000 mAh Lithium Polymer Battery,11499.0,redmi,Green,6.53,HD+,5.0,13.0,128,6
1086,2161,"REDMI 9 Activ (Metallic Purple, 128 GB)",6 GB RAM,128 GB RO,16.59 cm (6.53 inch) HD+ Display,13MP + 2MP | 5MP Front Camera,5000 mAh Lithium Polymer Battery,11499.0,redmi,Purple,6.53,HD+,5.0,13.0,128,6
1089,2169,"REDMI 9 Activ (Carbon Black, 128 GB)",6 GB RAM,128 GB RO,16.59 cm (6.53 inch) HD+ Display,13MP + 2MP | 5MP Front Camera,5000 mAh Lithium Polymer Battery,11499.0,redmi,Carbon,6.53,HD+,5.0,13.0,128,6


,Unnamed: 0,Model,Ram,Storage,Display,Camera,Battery,Price,Brand,Color,Display_Size,Display_Type,Front_Cam,Rear_Cam,ROM,RAM
